# Enunciado
Hola ingeniero 👋. Bienvenido a tu quinto desafio! Ya sabes que en [Vault-Tec Corporation](https://fallout.fandom.com/es/wiki/Vault-Tec_Corporation) tenemos sensores encargados de monitorear las condiciones actuales de nuestros refugios. En ocasiones pasadas detectamos que varios de ellos presentaban fallas recurrentes, impidiendo que pudieramos determinar las condiciones de nuestros bóvedas y darle la seguridad esperada a nuestros clientes en un mundo post apocaliptico 🙃. 

Anteriormente, a pedido de nuestro equipo, solucionamos algunos errores detectados en el funcionamiento de nuestro codigo; ademas, generamos pruebas unitarias.

Luego de tener las bases correctamente montadas (codigo + pruebas uitarias), nuestro equipo nos ha pedido indagar en un tema importantisimo: tratar de mejorar el rendimiento de nuestro codigo, tambien conocido como benchmarking. 

Uno de los desarrolladores Senior nos ha sugerido hacer pruebas de rendimiento usando el modulo Numpy. Para ello nos ha indicado que modifiquemos todas las partes del codigo que usan listas nativas de Python por **arrays de Numpy.**

# Insumos
Tu equipo te ha dado el siguiente codigo el cual deberas modificar. 

**IMPORTANTE:** En la vida real lo comun es que pases mas tiempo leyendo codigo de otros, que escribiendolo. 

In [1]:
from datetime import datetime
from typing import Literal

from pydantic import BaseModel


class Log(BaseModel):
    """Representa un registro (log) generado por un sensor.

    Atributos:
        sensor_id (Literal): Identificador del sensor que generó el registro.
            Valores posibles: "oxy_guard", "thermo_core", "hidra_flow", "radi_shield".
        event_type (Literal): Tipo de evento registrado.
            Valores posibles: "FAILURE_START" (inicio de falla) o "FAILURE_END" (fin de falla).
        timestamp (datetime): Fecha y hora exacta en que ocurrió el evento.
        duration_seconds (int): Duración de la falla en segundos. En caso de ser "FAILURE_START",
            puede ser 0 si aún no ha finalizado la falla.
    """

    sensor_id: Literal["oxy_guard", "thermo_core", "hidra_flow", "radi_shield"]
    event_type: Literal["FAILURE_END", "FAILURE_START"]
    timestamp: datetime
    duration_seconds: int

In [2]:
import json
from datetime import date
from os.path import abspath


class LogExtractor:
    """Clase para extraer y filtrar logs desde un archivo JSON."""

    def __init__(self, file_path: str):
        """
        Inicializa el extractor de logs con la ruta del archivo.

        Args:
            file_path (str): Ruta del archivo JSON que contiene los logs.
        """
        self.file_abspath = abspath(file_path)

    def from_file(self) -> list["Log"]:
        """
        Carga los logs desde el archivo JSON especificado en la inicialización.

        Returns:
            list[Log]: Lista de instancias de Log extraídas del archivo.
        """
        with open(self.file_abspath) as file:
            raw_logs: list[dict] = json.load(file)
        logs: list[Log] = [Log(**log) for log in raw_logs]
        return logs

    @staticmethod
    def fetch_logs_on_date(
        target_date: date, logs: list["Log"], sensor_id: int | None = None
    ) -> list["Log"]:
        """
        Filtra los logs que corresponden a una fecha y, opcionalmente, a un sensor específico.

        Args:
            target_date (date): Fecha objetivo para filtrar los logs.
            logs (list[Log]): Lista de logs disponibles.
            sensor_id (Optional[int], optional): ID del sensor para filtrar.
                Si es None, se incluyen logs de todos los sensores. Defaults to None.

        Returns:
            list[Log]: Lista de logs que cumplen con los filtros aplicados.
        """
        logs_on_date: list[Log] = list(
            filter(
                lambda x: (
                    x.timestamp.date() == target_date
                    and (sensor_id is None or x.sensor_id == sensor_id)
                ),
                logs,
            )
        )
        return logs_on_date

In [3]:
from datetime import date, timedelta


class SensorFailureAnalyzer:
    """Clase para analizar fallos de sensores a partir de registros de eventos.

    Esta clase permite calcular la duración de fallos de sensores,
    la probabilidad de fallo semanal y la probabilidad condicional
    de que un sensor falle dado que otro sensor también falló.
    """

    WEEK_TOTAL_SECONDS: int = 7 * 24 * (60**2)
    """int: Número total de segundos en una semana."""

    def __init__(self, logs: list[Log]):
        """Inicializa el analizador con los registros de logs.

        Args:
            logs (list[Log]): Lista de objetos `Log` que contienen los eventos de los sensores.
        """
        self.logs = logs

    def get_sensor_failure_duration(self, sensor_id: str, searched_date: date) -> float | None:
        """Obtiene la duración total del fallo de un sensor en una fecha específica.

        Busca en los logs los eventos `FAILURE_START` y `FAILURE_END`
        del sensor en la fecha indicada y calcula el tiempo total en segundos.

        Args:
            sensor_id (str): Identificador del sensor a analizar.
            searched_date (date): Fecha para la que se quiere obtener la duración del fallo.

        Returns:
            float | None: Duración del fallo en segundos.
            Devuelve `None` si no hay logs del sensor en esa fecha.
        """
        logs_on_date: list[Log] = LogExtractor.fetch_logs_on_date(
            sensor_id=sensor_id, target_date=searched_date, logs=self.logs
        )

        if not logs_on_date:
            return None

        failure_start_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_START"), 0
        )
        failure_end_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_END"), 0
        )

        return abs(failure_end_time - failure_start_time)

    def get_weekly_sensor_failure_duration(
        self, sensor_id: int, start_date: date, end_date: date | None = None
    ) -> list[dict]:
        """Obtiene la duración diaria de fallos de un sensor durante una semana.

        Args:
            sensor_id (int): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.
            end_date (Optional[date], optional): Fecha final del periodo.
                Si no se proporciona, se toma `start_date + 7 días`.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_seconds" (float | None): Duración del fallo en segundos.
        """
        end_date = start_date + timedelta(7) if end_date is None else end_date
        searched_week: list[date] = [
            (start_date + timedelta(days=i)) for i in range((end_date - start_date).days)
        ]
        return [
            {
                "sensor_id": sensor_id,
                "date": str(date),
                "failure_seconds": self.get_sensor_failure_duration(
                    sensor_id=sensor_id, searched_date=date
                ),
            }
            for date in searched_week
        ]

    def get_weekly_sensor_failure_probability(self, sensor_id: int, start_date: date) -> list[dict]:
        """Calcula la probabilidad diaria de fallo de un sensor durante una semana.

        La probabilidad se calcula como la proporción del tiempo en fallo
        respecto al total de segundos en una semana.

        Args:
            sensor_id (int): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_probability" (str): Probabilidad del fallo (0 a 1) con 4 decimales.
        """
        weekly_sensor_failure_duration = self.get_weekly_sensor_failure_duration(
            sensor_id, start_date
        )
        return [
            {
                "sensor_id": sensor_id,
                "date": x["date"],
                "failure_probability": f"{x['failure_seconds'] / self.WEEK_TOTAL_SECONDS:.4f}",
            }
            for x in weekly_sensor_failure_duration
        ]

    def get_conditional_failure_probability(
        self,
        sensor_a_id: int,
        sensor_b_id: int,
        start_date: date,
        failure_minutes_threshold: float | None = 3,
    ) -> float:
        """Calcula la probabilidad condicional de que un sensor falle dado que otro sensor falló.

        Se considera que un sensor falló en un día si su duración de fallo
        es mayor o igual a un umbral en minutos.

        Fórmula:
            P(A|B) = (Número de días donde A y B fallaron) / (Número de días donde B falló)

        Args:
            sensor_a_id (int): Identificador del sensor A (sensor condicionado).
            sensor_b_id (int): Identificador del sensor B (sensor condicionante).
            start_date (date): Fecha de inicio del periodo.
            failure_minutes_threshold (Optional[float], optional): Umbral de fallo en minutos.
                Defaults a 3 minutos.

        Returns:
            float: Probabilidad condicional P(A|B) como un valor entre 0 y 1.
            Devuelve 0 si B nunca falló en el periodo.
        """
        failure_seconds_threshold = failure_minutes_threshold * 60

        sensor_b_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_b_id, start_date)
        count_sensor_b_failures = sum(
            1
            for x in sensor_b_weekly_failures
            if x["failure_seconds"] and x["failure_seconds"] >= failure_seconds_threshold
        )

        if count_sensor_b_failures == 0:
            return 0.0

        sensor_a_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_a_id, start_date)
        count_sensor_a_b_failures = sum(
            1
            for x, y in zip(sensor_a_weekly_failures, sensor_b_weekly_failures, strict=False)
            if x["failure_seconds"]
            and y["failure_seconds"]
            and x["failure_seconds"] >= failure_seconds_threshold
            and y["failure_seconds"] >= failure_seconds_threshold
        )

        return count_sensor_a_b_failures / count_sensor_b_failures

# Tu turno!

Ahora es tu turno! Evaluemos el rendimiento de nuestro codigo actual contra la modificacion sugerida por nuestro compañero de equipo.

## 1. Benchmarking de LogExtractor
Reescribamos, probemos y comparemos los tiempos de ejecucion de la clase `LogExtractor` que tenemos en este momento, contra una version de esta clase modificada para usar vectores de Numpy.

**Reescribiendo la clase LogExtractor:** Reescribe la clase LogExtractor para que use vectores de Numpy en lugar de listas nativas de Python
1. Nombra la nueva clase NumpyLogExtractor. 
2. Modifica las funciones de esta clase que retornan `list[Log]` para que retornen `numpy.ndarray`.

- Consejo: Puedes prescindir de la clase Log. Busca sobre como representar un objeto en numpy.
    ```python
    log_dtype: list[tuple] = [
            ("sensor_id", "i4"),
            ("date", "M8[D]"),  # fecha con resolución de días
            ("event_type", "U16"), 
            ("duration_seconds", "f8"),
        ]
    ```

In [ ]:
# Tu codigo aqui

Comparemos el tiempo de ejecucion de nuestra clase LogExtractor pasada contra esta nueva implementacion en Numpy para la funcion `fetch_logs_on_date`.
- Consejo 1: Extrae los logs con ambas clases en una celda diferente a donde vayas a ejecutar `fetch_logs_on_date`; de no hacerlo, la medida del tiempo de ejecucion se vera contaminada con el tiempo tomado por la funcion de extraccion de logs.
- Consejo 2: Busca sobre magics proporcionadas por Jupyter para medicion de tiempos y como usarlas. Consulta sobre `%%timeit`.

In [ ]:
# Tu codigo aqui

2.09 μs ± 486 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## 2. Benchmarking de SensorFailureAnalyzer

Reescribamos, probemos y comparemos los tiempos de ejecucion de la clase `SensorFailureAnalyzer` que tenemos en este momento, contra una version de esta clase modificada para usar vectores de Numpy.

**Reescribiendo la clase SensorFailureAnalyzer:** Reescribe la clase SensorFailureAnalyzer para que use vectores de Numpy en lugar de listas nativas de Python
1. Nombra la nueva clase NumpySensorFailureAnalyzer. 

In [ ]:
# Tu codigo aqui

Comparemos el tiempo de ejecucion de nuestra clase SensorFailureAnalyzer pasada contra esta nueva implementacion en Numpy para la funcion `get_weekly_sensor_failure_probability`.
- Consejo 1: Usa los logs anteriormente extraidos e instancia (crea) ambas clases en una celda diferente a donde vayas a ejecutar `get_weekly_sensor_failure_probability`; de no hacerlo, la medida del tiempo de ejecucion se vera contaminada con el tiempo tomado para crear las clases.

In [ ]:
# Tu codigo aqui